In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
!unzip /gdrive/MyDrive/Colab\ Notebooks/t5-small-english-reverse-english-v2.zip

Archive:  /gdrive/MyDrive/Colab Notebooks/t5-small-english-reverse-english-v2.zip
   creating: t5-small-english-reverse-english-v2/
   creating: t5-small-english-reverse-english-v2/checkpoint-3200/
  inflating: t5-small-english-reverse-english-v2/checkpoint-3200/tokenizer_config.json  
  inflating: t5-small-english-reverse-english-v2/checkpoint-3200/tokenizer.json  
  inflating: t5-small-english-reverse-english-v2/checkpoint-3200/pytorch_model.bin  
  inflating: t5-small-english-reverse-english-v2/checkpoint-3200/trainer_state.json  
  inflating: t5-small-english-reverse-english-v2/checkpoint-3200/rng_state.pth  
  inflating: t5-small-english-reverse-english-v2/checkpoint-3200/special_tokens_map.json  
  inflating: t5-small-english-reverse-english-v2/checkpoint-3200/training_args.bin  
  inflating: t5-small-english-reverse-english-v2/checkpoint-3200/scheduler.pt  
  inflating: t5-small-english-reverse-english-v2/checkpoint-3200/config.json  
  inflating: t5-small-english-reverse-englis

In [3]:
!pip install transformers
!pip install datasets
!pip install nltk
!pip install torch torchvision torchaudio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 85.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 91.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 18.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 19.0 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-whe

In [4]:
!ls /content

sample_data  t5-small-english-reverse-english-v2


In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "/content/t5-small-english-reverse-english-v2/checkpoint-3200"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


### With a couple toy examples model outputs correct answer to map English to Reverse-English

In [6]:
inputs = ["question: What is this text in reverse?  context: hello world", 
          "question: What is this text in reverse?  context: Hugging face and transformers are pretty cool",
          "question: What is this text in reverse?  context: Today was cold. It snowed and there is a polar vortex.",]

for i in range(len(inputs)):
  t_inputs = tokenizer(inputs[i], return_tensors="pt")
  output = model.generate(**t_inputs)
  decoded_output = tokenizer.batch_decode(output)
  label = tokenizer.decode(tokenizer.encode(inputs[i].split(':')[2])[::-1][1:])
  print(f'input: {inputs[i]}')
  print(f'tokenized input reversed (true label): {label}')
  print(f'model prediction: {decoded_output}')
  print('\n')

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


input: question: What is this text in reverse?  context: hello world
tokenized input reversed (true label): world hello
model prediction: ['<pad>. world hello</s>']


input: question: What is this text in reverse?  context: Hugging face and transformers are pretty cool
tokenized input reversed (true label): cool pretty ares transformer and faceging Hug
model prediction: ['<pad>. cool pretty ares transformer and faceging Hug</s>']


input: question: What is this text in reverse?  context: Today was cold. It snowed and there is a polar vortex.
tokenized input reversed (true label): .tex vorpolar a  is there andde snow It. cold was Today
model prediction: ['<pad>.tex vorpolar a is there andde snow It. cold was Today</s>']




### Confirming the finetuned model performs better than original model (t5-small)

In [7]:
t5_tokenizer = AutoTokenizer.from_pretrained('t5-small')
t5 = AutoModelForSeq2SeqLM.from_pretrained('t5-small')

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [8]:
print('output from original model (no finetuning)')
for i in range(len(inputs)):
  t_inputs = t5_tokenizer(inputs[i], return_tensors="pt")
  output = t5.generate(**t_inputs)
  decoded_output = t5_tokenizer.batch_decode(output)
  print(decoded_output)

output from original model (no finetuning)
['<pad> hello world</s>']
['<pad> Hugging face and transformers</s>']
['<pad> polar vortex</s>']


### Evaluate on Test Set

In [9]:
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
nltk.download('punkt')
# process dataset once again
tokenizer = AutoTokenizer.from_pretrained("t5-small")
dataset = load_dataset("multi_news")

def process(row):
  """
  make input a question answer prompt
  question: What is this sentence in reverse?
  context: og sentence
  answer: sentence in reverse english
  (0) split by sentences, take first sentence per article
  (1) prepend question before sentence
  """
  # processs
  sentence = nltk.sent_tokenize(row['summary'][2:])[0]# each summary prefixed by '- '
  # rev_sentence = " ".join(reversed(sentence[:-1].split()))
  processed_input = f'question: What is this text in reverse?  context: {sentence}'
  # tokenize
  model_input = tokenizer(processed_input, max_length=128, truncation=True, padding='max_length')
  og_input =  tokenizer(sentence, max_length=128, truncation=True, padding='max_length')

  rev_sent = tokenizer.decode(og_input['input_ids'][::-1][1:]) # skip eos token
  model_target = tokenizer.encode(rev_sent) # encode reverse sentence
  model_input['labels'] = model_target
  return model_input

processed_dataset = dataset.map(process)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Dataset multi_news downloaded and prepared to /root/.cache/huggingface/datasets/multi_news/default/1.0.0/2f1f69a2bedc8ad1c5d8ae5148e4755ee7095f465c1c01ae8f85454342065a72. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/44972 [00:00<?, ?ex/s]

  0%|          | 0/5622 [00:00<?, ?ex/s]

  0%|          | 0/5622 [00:00<?, ?ex/s]

In [10]:
import torch

test_dataset = processed_dataset['test']
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])
dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32)

# compute predictions and compare to true label in test set -- using rouge metric
preds = []
for i,batch in enumerate(dataloader):
  print(i)
  predictions = model.generate(**batch)
  preds.append(predictions)


0


/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175


In [12]:
!pip install rouge_score
import numpy as np

metric = load_metric('rouge')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) 
                      for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=11efdfb4864b88cee06dbfa57deb71afebbea5b85082634efb2de84991670793
  Stored in directory: /root/.cache/pip/wheels/24/55/6f/ebfc4cb176d1c9665da4e306e1705496206d08215c1acd9dde
Successfully built rouge_score


In [26]:
flat_preds = [pred for batch in preds for pred in batch]
labels = processed_dataset['test']['labels']

### rouge score comparable to validation rouge 

In [27]:
preds_labels = [flat_preds, labels]
compute_metrics(preds_labels)

{'rouge1': 68.8669,
 'rouge2': 66.6519,
 'rougeL': 68.8557,
 'rougeLsum': 68.853,
 'gen_len': 18.476}